In [1]:
import random
from bs4 import BeautifulSoup as bs
import time
import requests
import re
import numpy as np
import pandas as pd

In [2]:
url = requests.get('https://myanimelist.net/anime/season/archive')
soup = bs(url.text,"html5lib")


In [3]:
#picking a random set of 96 seasons to search th
seasons = soup.find_all('td')
randindex = random.sample(range(0, 150), 80)
seasons_to_pick = [seasons[x] for x in randindex]
seasons_to_pick

[<td>\n                                              <a href="https://myanimelist.net/anime/season/1982/spring">\n                  Spring 1982\n                </a>\n                          </td>,
 <td>\n                                              <a href="https://myanimelist.net/anime/season/1996/summer">\n                  Summer 1996\n                </a>\n                          </td>,
 <td>\n                                              <a href="https://myanimelist.net/anime/season/2010/fall">\n                  Fall 2010\n                </a>\n                          </td>,
 <td>\n                                              <a href="https://myanimelist.net/anime/season/1981/fall">\n                  Fall 1981\n                </a>\n                          </td>,
 <td>\n                                              <a href="https://myanimelist.net/anime/season/1983/summer">\n                  Summer 1983\n                </a>\n                          </td>,
 <td>\n 

Iterating through every show in a given season

In [19]:
"""fine tune the lookup list"""
def getText(soup,df):
    testdf = pd.Series(index=['Name','Studio','Episodes','Genres','Source','Type','Score','Members','Season','Year'])
    for index, s in enumerate(soup.find_all('div',class_=re.compile(r'seasonal-anime-list js-seasonal-anime-list js-seasonal-anime-list-key'))):
        #grab each show in the above category
        if index is 1:
            continue
        else:
            #print (s.find_all('div',class_="seasonal-anime js-seasonal-anime")),index
            #print (soup.find('title').text.strip().split(' ')[0]), (soup.find('title').text.strip().split(' ')[1])
            for anime in s.find_all('div',class_="seasonal-anime js-seasonal-anime"):
                testdf.Season = (soup.find('title').text.strip().split(' ')[0])
                testdf.Year = (soup.find('title').text.strip().split(' ')[1])
                testdf.Name = anime.find('a',{'class' : 'link-title'}).text.strip()
                testdf.Studio = anime.find('span',{"class":'producer'}).text.strip()
                testdf.Episodes = anime.find('div',{'class':'eps'}).text.strip().split(' ')[0]
                testdf.Source = anime.find('span',{"class":'source'}).text.strip()
                testdf.Type = re.search("[a-zA-Z]+",str(anime.find('div',{'class':'info'}).text)).group(0)
                testdf.Score = anime.find('span',{'class':'score'}).text.strip()
                testdf.Genres = [x.text.strip() for x in anime.find_all('span',class_='genre')]
                testdf.Members= anime.find('span',class_='member fl-r').text.strip()
                df = df.append(testdf,ignore_index=True)
    return df

Iterating through random seasons

In [20]:
df = pd.DataFrame(columns=['Name','Studio','Episodes','Genres','Source','Type','Score','Members','Season','Year'])
time_avg = []
for index,s in enumerate(seasons_to_pick):
    t = time.time()
    try:
        url = requests.get(s.find('a').get('href'), timeout=.9,verify=True,stream=False)
        print '{} time to grab page is {}'.format(index,time.time() - t)
        time_avg.append(time.time() - t)
        soup = bs(url.text)
        df = getText(soup,df)
    except requests.exceptions.Timeout:
        print 'taking too long, skipping'
print np.average(time_avg)

0 time to grab page is 0.712000131607
1 time to grab page is 1.00800013542
taking too long, skipping
3 time to grab page is 0.901999950409
4 time to grab page is 0.875
5 time to grab page is 1.1930000782
6 time to grab page is 0.974999904633
7 time to grab page is 1.39199995995
8 time to grab page is 0.700999975204
9 time to grab page is 0.84299993515
10 time to grab page is 1.28600001335
11 time to grab page is 1.22900009155
12 time to grab page is 1.16400003433
13 time to grab page is 0.732999801636
14 time to grab page is 1.28099989891
15 time to grab page is 0.968999862671
16 time to grab page is 0.816999912262
17 time to grab page is 1.29200005531
18 time to grab page is 0.81200003624
19 time to grab page is 1.26800012589
taking too long, skipping
21 time to grab page is 0.892999887466
22 time to grab page is 1.40600013733
taking too long, skipping
taking too long, skipping
25 time to grab page is 1.3860001564
26 time to grab page is 0.778000116348
27 time to grab page is 1.280000

In [21]:
df

,Name,Studio,Episodes,Genres,Source,Type,Score,Members,Season,Year
0,Patalliro!,Toei Animation,49,"[Adventure, Comedy, Romance, Shoujo, Shounen Ai]",Manga,TV,7.62,"2,748",Spring,1982
1,Mahou no Princess Minky Momo,Production Reed,63,"[Magic, Shoujo]",Original,TV,6.77,"2,524",Spring,1982
2,Kikou Kantai Dairugger XV,Toei Animation,52,"[Action, Adventure, Mecha, Sci-Fi, Shounen]",-,TV,6.56,"1,228",Spring,1982
3,Ochamegami Monogatari: Korokoro Pollon,-,46,"[Parody, Magic, Comedy]",Manga,TV,7.13,"1,169",Spring,1982
4,Tezuka Osamu no Don Dracula,-,8,"[Comedy, Horror, Supernatural, Vampire]",Manga,TV,6.38,"1,057",Spring,1982
5,Tondera House no Daibouken,Tatsunoko Production,52,"[Adventure, Historical, Sci-Fi]",Original,TV,6.50,681,Spring,1982
6,Kagaku Kyuujo-tai TechnoVoyager,Toei Animation,18,"[Action, Adventure, Mecha, Sci-Fi, Space]",-,TV,6.44,432,Spring,1982
7,Makyou Densetsu Acrobunch,-,24,"[Mecha, Sci-Fi, Shounen]",-,TV,6.47,352,Spring,1982
8,Anime Yasei no Sakebi,-,22,"[Adventure, Drama]",-,TV,N/A,206,Spring,1982
9,Haguregumo,"Madhouse, Toei Animation",1,"[Comedy, Historical, Seinen]",-,Movie,N/A,252,Spring,1982


this is a test function

In [17]:
#find all types of animes in a given season. i.e. TV(new), TV(cont), OVA etc
dftest = pd.DataFrame(columns=['Name','Studio','Episodes','Genres','Source','Type','Score','Members','Season','Year'])
testdf = pd.Series(index=['Name','Studio','Episodes','Genres','Source','Type','Score','Members','Season','Year'])
for index, s in enumerate(soup.find_all('div',class_=re.compile(r'seasonal-anime-list js-seasonal-anime-list js-seasonal-anime-list-key'))):
    #grab each show in the above category
    if index is 1:
        continue
    else:
        #print (s.find_all('div',class_="seasonal-anime js-seasonal-anime")),index
        #print (soup.find('title').text.strip().split(' ')[0]), (soup.find('title').text.strip().split(' ')[1])
        for anime in s.find_all('div',class_="seasonal-anime js-seasonal-anime"):
            testdf.Season = (soup.find('title').text.strip().split(' ')[0])
            testdf.Year = (soup.find('title').text.strip().split(' ')[1])
            testdf.Name = anime.find('a',{'class' : 'link-title'}).text.strip()
            testdf.Studio = anime.find('span',{"class":'producer'}).text.strip()
            testdf.Episodes = anime.find('div',{'class':'eps'}).text.strip().split(' ')[0]
            testdf.Source = anime.find('span',{"class":'source'}).text.strip()
            testdf.Type = re.search("[a-zA-Z]+",str(anime.find('div',{'class':'info'}).text)).group(0)
            testdf.Score = anime.find('span',{'class':'score'}).text.strip()
            testdf.Genres = [x.text.strip() for x in anime.find_all('span',class_='genre')]
            testdf.Members= anime.find('span',class_='member fl-r').text.strip()
            dftest = dftest.append(testdf,ignore_index=True)


In [22]:
df.to_csv('randomselection.csv',encoding='utf-8')

df = pd.read_csv('randomselection.csv')

del df[df.columns[0]]
df.Score =  df.Score.fillna(np.mean(df.Score))

df.to_csv('cleaned.csv',encoding='utf-8')

